# setup

In [ ]:
library(broom)
# library(ggExtra)
library(tidyverse)

In [ ]:
gsutil = function(text, echo = FALSE) {
  text = glue::glue('gsutil ', text)
  if (echo) base::print(text)
  system(command = text, intern = TRUE)
}

gscp = function(src, dest = '.', echo = TRUE) {
    gsutil(paste('cp', src, dest))
}

gsls = function(path, echo = TRUE) {
    gsutil(paste('ls', path))
}

In [ ]:
lu = function(x) (length(unique(x)))

In [ ]:
years_between = function(a, b) {
    as.numeric(difftime(b, a, units = 'weeks')/52.25)
}

In [ ]:
is_not_na = function(x) (!is.na(x))

In [ ]:
# read in all AGD data
'AGD250kcohortshiftedsampledates.csv' |>
read_csv(show_col_types = FALSE, 
         name_repair = ~str_replace_all(., ' ', '_')) |>
filter(!is.na(Data_Available)) |>
mutate(biosample_date = mdy(SHIFTED_SAMPLE_DATE)) |>
select(-SHIFTED_SAMPLE_DATE, -Data_Available) |>
# join in sample ID
left_join(read_csv('AGD_DATA_SAMPLE_ID_MAP_Q3_2024.csv', show_col_types = FALSE)) |>
select(-OUTPUT_FOLDER_ID) |>
# join in demographics
left_join(read_csv('demog_of_agd.csv', show_col_types = FALSE)) |>
# filtering join for DTS data
# semi_join(read_tsv('targeted_sequencing_biovu_rad_mtp_kidney_yp_03182025.tsv', show_col_types = FALSE)) |>
# mutate(age_at_biosample = years_between(birth_date, biosample_date)) |>
filter(GRID %in% read_rds('all_dts_grids.rds')) |>
glimpse() ->
cohort_dts_wgs

In [ ]:
cohort_dts_wgs$GRID |> lu()

In [ ]:
cohort_dts_wgs$person_id |> lu()

In [ ]:
cohort_dts_wgs |> distinct(person_id, biosample_date) |> glimpse()

# read in and prep targeted seq calls

In [ ]:
'targeted_sequencing_biovu_rad_mtp_kidney_yp_03182025.tsv' |>
read_tsv(show_col_types = FALSE) |> 
transmute(
    GRID,
    cohort,
    biosample_date = mdy(biovu_sample_date),
    chip_gene_dts = Gene.refGene,
#     chip_mut_func_dts = Func.refGene,
#     chip_mut_exofunc_dts = ExonicFunc.refGene,
    chip_mut_aachange_dts = NonsynOI,
    AF_dts = AF
) |>
# glimpse() |>
# use only DTS data from same sample as WGS data
semi_join(cohort_dts_wgs, by = join_by(GRID, biosample_date)) |>
select(-biosample_date) |>
# glimpse() |>
# remove duplicates
distinct(GRID, chip_gene_dts, chip_mut_aachange_dts, .keep_all = TRUE) |>
# only CHIP calls (don't need a row for everyone)
filter(!is.na(chip_gene_dts)) |>
glimpse() ->
chip_calls_dts_noheart

In [ ]:
read_tsv('chip_calls_hearttransplant.tsv', show_col_types = FALSE) |> 
filter(!is.na(chip_gene_dts)) |> 
semi_join(cohort_dts_wgs, by = 'GRID') |>
mutate(cohort = 'heart') |>
glimpse() ->
chip_calls_heart

In [ ]:
bind_rows(
    chip_calls_dts_noheart,
    chip_calls_heart
) |>
glimpse() ->
chip_calls_dts

In [ ]:
chip_calls_dts$GRID |> lu()

In [ ]:
mean(chip_calls_dts$GRID %in% cohort_dts_wgs$GRID)

# read in and prep WGS calls

In [ ]:
filter_wgs_chip_calls <- function(data, 
                                  min_dp = 20, 
                                  min_ad2 = 2,
                                  require_both_strands = TRUE,
                                  max_age_p = 0.1,
                                  max_tert_p = 0.1,
                                  max_binom_p = 0.01) {

    data |>
    filter(
        DP_wgs >= min_dp,
        AD2 >= min_ad2,
        (!require_both_strands | both_strands),
        (is.na(hotspot_age_p_val) | hotspot_age_p_val <= max_age_p | hotspot_tert_p_val <= max_tert_p),
        binom_p_val <= max_binom_p
    )
}

In [ ]:
'putative_chip_vars_flagged_nofilters.RDS' |>
read_rds() |>
transmute(
    SAMPLE_ID = as.numeric(Sample),
    chr = Chr,
    start = Start,
    end = End,
    chip_gene_wgs = Gene.refGene,
    chip_mut_func_wgs = Func.refGene,
    chip_mut_exofunc_wgs = ExonicFunc.refGene,
    chip_mut_aachange_wgs = NonsynOI,
    DP_wgs = DP,
    AD_wgs = AD,
    AF_wgs = AF,
    # there are none here
    # multi_allelic_wgs = multi_allelic,
    both_strands,
    # no multiallelic so not necessary
    # maxAF_wgs = maxAF
    hotspot_age_p_val,
    hotspot_tert_p_val,
    binom_p_val
) |>
# filter to only GRIDs that also have DTS
inner_join(cohort_dts_wgs, by = 'SAMPLE_ID') |>
# some GRIDs were run through mutect twice -- keep only one
distinct(GRID, chip_gene_wgs, chip_mut_aachange_wgs, AF_wgs, .keep_all = TRUE) |>
# make AD2 computable
separate_wider_delim(cols = 'AD_wgs', delim = ',', names = c('AD1', 'AD2')) |>
# apply standard filters
filter_wgs_chip_calls() |>
# glimpse() |>
# keep only needed columns
select(GRID, starts_with('chip_'), AF_wgs, DP_wgs) |>
glimpse() ->
chip_calls_wgs

In [ ]:
chip_calls_wgs$GRID |> lu()

In [ ]:
mean(chip_calls_wgs$GRID %in% cohort_dts_wgs$GRID)

# cohort basics

In [ ]:
# size of study cohort?
nrow(cohort_dts_wgs)

In [ ]:
# how many have CHIP by gold standard?
nrow(chip_calls_dts |> distinct(GRID))

In [ ]:
# what % have CHIP?
round(100 * nrow(distinct(chip_calls_dts, GRID))/nrow(cohort_dts_wgs))

In [ ]:
# how many have single CHIP vs multi-CHIP?
chip_calls_dts |>
filter(!is.na(chip_gene_dts)) |>
count(GRID) |>
count(n, name = 'num_chip_muts')

# sens/spec by person

In [ ]:
cohort_dts_wgs |>
left_join(
    chip_calls_dts |>
    group_by(GRID) |>
    summarise(AF_dts = max(AF_dts))
) |>
left_join(
    chip_calls_wgs |>
    group_by(GRID) |>
    summarise(AF_wgs = max(AF_wgs))
) |>
glimpse() ->
chip_calls_joined_byperson

In [ ]:
chip_calls_joined_byperson |>
filter(is.na(AF_dts)) |>
pull(AF_wgs) |>
is.na() |> mean() |> print() ->
wgs_spec

In [ ]:
chip_calls_joined_byperson |>
filter(!is.na(AF_dts)) |>
pull(AF_wgs) |>
is_not_na() |> mean() |> print() ->
wgs_sens

In [ ]:
chip_calls_joined_byperson |>
filter(!is.na(AF_dts)) |>
filter(AF_dts > 0.05) |>
pull(AF_wgs) |>
is_not_na() |> mean() |> print() ->
wgs_sens_gt_05

In [ ]:
chip_calls_joined_byperson |>
filter(!is.na(AF_dts)) |>
filter(AF_dts > 0.1) |>
pull(AF_wgs) |>
is_not_na() |> mean() |> print() ->
wgs_sens_gt_10

In [ ]:
tibble(
    `specificity` = wgs_spec,
    `sensitivity for people\nwith CHIP` = wgs_sens,
    `sensitivity for people\nwith AF > 5%` = wgs_sens_gt_05,
    `sensitivity for people\nwith AF > 10%` = wgs_sens_gt_10) |>
pivot_longer(cols = everything()) |>
print() |>
mutate(name = factor(name, levels = name)) |>
ggplot(aes(x = name, y = value)) +
geom_bar(stat = 'identity') +
theme_bw() +
labs(x = NULL, y = 'sensitivity or specificity')

In [ ]:
ggsave('wgs_sens_spec_byperson.pdf', height = 4, width = 6)

# sens/spec by variant

In [ ]:
chip_calls_dts |>
full_join(chip_calls_wgs, 
          by = join_by(
              GRID, 
              chip_gene_dts == chip_gene_wgs,
              chip_mut_aachange_dts == chip_mut_aachange_wgs)) |>
full_join(cohort_dts_wgs, by = 'GRID') |>
glimpse() ->
chip_calls_joined_byvariant

In [ ]:
chip_calls_joined_byvariant |>
filter(is.na(AF_dts)) |>
pull(AF_wgs) |>
is.na() |> mean() |> print() ->
wgs_spec

In [ ]:
chip_calls_joined_byvariant |>
filter(is.na(AF_dts), !is.na(AF_wgs)) |>
# glimpse()
# ggplot(aes(x = age_at_biosample, y = AF_wgs)) + geom_point()
# ggplot(aes(x = AF_wgs)) + geom_histogram()
pull(AF_wgs) ->
false_pos_AF_wgs

In [ ]:
# for use in sims
# dput(false_pos_AF_wgs)

In [ ]:
chip_calls_joined_byvariant |>
filter(!is.na(AF_dts)) |>
pull(AF_wgs) |>
is_not_na() |> mean() |> print() ->
wgs_sens

In [ ]:
chip_calls_joined_byvariant |>
filter(!is.na(AF_dts)) |>
filter(AF_dts > 0.05) |>
pull(AF_wgs) |>
is_not_na() |> mean() |> print() ->
wgs_sens_gt_05

In [ ]:
chip_calls_joined_byvariant |>
filter(!is.na(AF_dts)) |>
filter(AF_dts > 0.1) |>
pull(AF_wgs) |>
is_not_na() |> mean() |> print() ->
wgs_sens_gt_10

In [ ]:
chip_calls_joined_byvariant |>
filter(!is.na(AF_dts)) |>
filter(AF_dts > 0.2) |>
pull(AF_wgs) |>
is_not_na() |>
mean() ->
wgs_sens_gt_20

In [ ]:
tibble(
    `specificity` = wgs_spec,
    `sensitivity for CHIP\nvariants` = wgs_sens,
    `sensitivity for variants\nwith AF > 5%` = wgs_sens_gt_05,
    `sensitivity for variants\nwith AF > 10%` = wgs_sens_gt_10) |>
pivot_longer(cols = everything()) |>
mutate(name = factor(name, levels = name)) |>
ggplot(aes(x = name, y = value)) +
geom_bar(stat = 'identity') +
scale_y_continuous(labels = scales::percent) +
theme_bw() +
labs(x = NULL, y = 'sensitivity or specificity')

In [ ]:
ggsave('wgs_sens_spec_byvariant.pdf', height = 4, width = 6)

In [ ]:
chip_calls_joined_byvariant |>
filter(AF_wgs != 0 & AF_dts != 0) |>
lm(AF_wgs ~ AF_dts, data = _) |>
tidy()

In [ ]:
chip_calls_joined_byvariant |>
filter(AF_wgs != 0 & AF_dts != 0) |>
# glimpse() 
ggplot(aes(x = AF_dts, y = AF_wgs)) +
geom_point(aes(color = factor(round(DP_wgs, -1)))) +
geom_abline(slope = 1, intercept = 0) +
# geom_abline(slope = 1, intercept = -0.05, color = 'blue') +
# geom_abline(slope = 1, intercept = 0.05, color = 'blue') +
geom_smooth(method = 'lm') +
theme_bw() +
xlim(c(0, 0.45)) +
ylim(c(0, 0.45)) +
labs(x = 'AF', y = 'WGS-estimated AF', color = 'depth') 

ggsave('AF_dotplot.pdf', height = 4, width = 5)

In [ ]:
chip_calls_joined_byvariant |>
filter(AF_wgs != 0 & AF_dts != 0) |>
mutate(AF_dev = abs(AF_dts - AF_wgs)) |>
lm(AF_dev ~ DP_wgs, data = _) |>
tidy()

In [ ]:
chip_calls_joined_byvariant |>
filter(AF_wgs != 0 & AF_dts != 0) |>
mutate(AF_dev = (AF_dts - AF_wgs)^2) |>
lm(AF_dev ~ DP_wgs, data = _) |>
tidy()